<a href="https://colab.research.google.com/github/kake01/Dialogue/blob/Vectorization/%E5%90%8D%E5%A4%A7%E3%81%AE%E3%83%99%E3%82%AF%E3%83%88%E3%83%AB%E5%8C%96(word2vec).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#名古屋大コーパスの前処理,ベクトル化

## 環境構築

In [0]:
pip install janome

In [0]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import re
import numpy as np
import pickle
from gensim.models import word2vec
from janome.tokenizer import Tokenizer

# 文章ごとのリストを作成

In [0]:
# ファイルの読み込み
with open("corpus.txt", mode="r", encoding="utf-8") as f:
  text = f.read()

seperator = "。"  # 。をセパレータに指定
sentence_list = text.split(seperator)  # セパレーターを使って文章をリストに分割する
sentence_list.pop() # 最後の要素は空の文字列になるので、削除
sentence_list = [x+seperator for x in sentence_list]  # 文章の最後に。を追加

In [0]:
print(sentence_list[0])

の町というのはちいちゃくって、城壁がこう町全体をぐるっと回ってて、それが城壁の上を歩いても１時間ぐらいですよね。


#分かち書きリスト

In [0]:
t = Tokenizer()
words = []
for sentence in sentence_list:
    words.append(t.tokenize(sentence, wakati=True))  # 文章ごとに単語に分割し、リストに格納

In [0]:
print(len(words))
print(words[0])

115173
['の', '町', 'という', 'の', 'は', 'ちい', 'ちゃ', 'くっ', 'て', '、', '城壁', 'が', 'こう', '町', '全体', 'を', 'ぐるっと', '回っ', 'て', 'て', '、', 'それ', 'が', '城壁', 'の', '上', 'を', '歩い', 'て', 'も', '１', '時間', 'ぐらい', 'です', 'よ', 'ね', '。']


In [0]:
# size : 中間層のニューロン数 デフォルト100
# min_count : この値以下の出現回数の単語を無視
# window : 対象単語を中心とした前後の単語数
# iter : epochs数
# sg : skip-gramを使うかどうか 0:CBOW 1:skip-gram
model = word2vec.Word2Vec(words, size=250, min_count=5, window=5, iter=20, sg = 0)

In [0]:
print(model.wv.vectors.shape)  # 分散表現の形状
print(model.wv.vectors)  # 分散表現
print(len(model.wv.index2word))  # 語彙の数
# (7452, 250)

In [0]:
print(model.wv.vectors[0])

#モデル,行列の保存

In [0]:
# # モデルを保存
model.save("word2vec.model")
# 行列を保存
with open('word2vec_vector.pickle', mode='wb') as f:  # pickleに保存
    pickle.dump(model.wv.vectors, f)

#モデル,行列を読み込む

In [0]:
# モデルを読み込む
word2vec_model = word2vec.Word2Vec.load("word2vec.model")
with open('word2vec_vector.pickle', 'rb') as web:
  word2vec_vector = pickle.load(web)
# print (word2vec_vector[0])

##encoder,decoderへの入力,decoderの正解を作成

In [0]:
seperator = "。"
sentence_list = text.split(seperator) 
sentence_list.pop() 
sentence_list = [x+seperator for x in sentence_list]

# センテンスで長すぎる文章の削除
max_sentence_length = 128
sentence_list = [sentence for sentence in sentence_list if len(sentence) <= max_sentence_length]  # 長すぎる文章のカット


# n_char = len(chars_list)  # 文字の種類の数   <= model.wv.index2word
n_char = len(model.wv.index2word)  # 文字の種類の数
n_sample = len(sentence_list) - 1  # サンプル数

x_sentences = []  # 入力の文章
t_sentences = []  # 正解の文章
for i in range(n_sample):
  x_sentences.append(sentence_list[i])
  t_sentences.append("\t" + sentence_list[i+1] + "\n")  # 正解は先頭にタブ、末尾に改行を加える
max_length_x = max_sentence_length  # 入力文章の最大長さ
max_length_t = max_sentence_length + 2  # 正解文章の最大長さ

# x_encoder = np.zeros((n_sample, max_length_x, n_char), dtype=np.bool)  # encoderへの入力
# x_decoder = np.zeros((n_sample, max_length_t, n_char), dtype=np.bool)  # decoderへの入力
# t_decoder = np.zeros((n_sample, max_length_t, n_char), dtype=np.bool)  # decoderの正解

# for i in range(n_sample):
#     x_sentence = x_sentences[i]
#     t_sentence = t_sentences[i]
#     for j, char in enumerate(x_sentence):
#         x_encoder[i, j, char_indices[char]] = 1  # encoderへの入力をone-hot表現で表す
#     for j, char in enumerate(t_sentence):
#         x_decoder[i, j, char_indices[char]] = 1  # decoderへの入力をone-hot表現で表す
#         if j > 0:  # 正解は入力より1つ前の時刻のものにする
#             t_decoder[i, j-1, char_indices[char]] = 1

In [0]:
print(x_encoder.shape)
# (114553, 128, 272)
# (行数, 入力文章の最大長さ, 文字の種類の数)

(114553, 128, 272)


##類似語のテスト

In [0]:
print(temp_model.wv.most_similar("怖い"))  # 最も似ている単語

[('つらい', 0.6198047399520874), ('セクシー', 0.609634280204773), ('うまい', 0.6051341891288757), ('かわいそう', 0.5984027981758118), ('きつい', 0.5945082902908325), ('かっこいい', 0.5899600982666016), ('やばい', 0.5897552371025085), ('おもしろい', 0.5887147784233093), ('厳しい', 0.5871392488479614), ('さみしい', 0.5859197378158569)]


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
